In [6]:
# Surpress warnings:
def warn(*args, **kwargs):
    pass

import warnings

warnings.warn = warn

In [7]:
import pandas as pd
import seaborn as sns
import numpy as np

from imblearn.over_sampling import RandomOverSampler, SMOTE
from sklearn.utils import shuffle
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.model_selection import train_test_split, learning_curve
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, accuracy_score, precision_recall_fscore_support, confusion_matrix,precision_score, recall_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, StackingClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC

from imblearn.under_sampling import RandomUnderSampler
from sklearn import metrics
from sklearn.inspection import permutation_importance
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from collections import Counter

from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import r2_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import scale
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.pipeline import Pipeline

In [8]:
ownership_df_class = pd.read_csv("../../data/final_data/final_ownership_data.csv")
sales_df_reg = pd.read_csv("../../data/final_data/final_sales_data.csv") 

In [9]:
ownership_df_class = ownership_df_class.drop(columns=['Unnamed: 0', '_c0'])
ownership_df_class

,Occupation,Annual_Income,Credit_Score,Years_of_Employment,Finance_Status,Car,Number_of_Children
0,Teacher,52800.0,750,10,Good,Yes,2
1,Engineer,79200.0,800,8,Good,Yes,1
2,Nurse,33000.0,650,5,Fair,No,0
3,Doctor,105600.0,900,12,Excellent,Yes,3
4,Businessman,132000.0,700,15,Good,Yes,2
...,...,...,...,...,...,...,...
391,Customer Support,4620.0,640,3,Stable,No,1
392,Dental Hygienist,6336.0,670,4,Stable,Yes,2
393,Project Manager,9240.0,730,5,Stable,Yes,0
394,Medical Assistant,4620.0,640,3,Stable,No,1


In [10]:
sales_df_reg = sales_df_reg.drop(columns=['Unnamed: 0', '_c0'])
sales_df_reg

,Car_id,Date,Gender,Annual_Income,Dealer_Name,Company,Model,Engine,Transmission,Color,Price,Dealer_No,Body_Style,Phone,Dealer_Region,Month,Year
0,C_CND_000001,2022-01-02,Male,148.5,Buddy Storbeck's Diesel Service Inc,Ford,Expedition,DoubleÂ Overhead Camshaft,Auto,Black,26000,06457-3834,SUV,8264678,Connecticut,1,2022
1,C_CND_000002,2022-01-02,Male,16280.0,C & M Motors Inc,Dodge,Durango,DoubleÂ Overhead Camshaft,Auto,Black,19000,60504-7114,SUV,6848189,Colorado,1,2022
2,C_CND_000003,2022-01-02,Male,11385.0,Capitol KIA,Cadillac,Eldorado,Overhead Camshaft,Manual,Red,31500,38701-8047,Passenger,7298798,South Carolina,1,2022
3,C_CND_000004,2022-01-02,Male,148.5,Chrysler of Tri-Cities,Toyota,Celica,Overhead Camshaft,Manual,Pale White,14000,99301-3882,SUV,6257557,Washington,1,2022
4,C_CND_000005,2022-01-02,Male,16115.0,Chrysler Plymouth,Acura,TL,DoubleÂ Overhead Camshaft,Auto,Red,24500,53546-9427,Hatchback,7081483,Wisconsin,1,2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23901,C_CND_023902,2023-12-31,Male,148.5,C & M Motors Inc,Plymouth,Voyager,Overhead Camshaft,Manual,Red,12000,60504-7114,Passenger,8583598,Washington,12,2023
23902,C_CND_023903,2023-12-31,Female,9900.0,Ryder Truck Rental and Leasing,Chevrolet,Prizm,DoubleÂ Overhead Camshaft,Auto,Black,16000,06457-3834,Hardtop,7914229,Connecticut,12,2023
23903,C_CND_023904,2023-12-31,Male,7755.0,Chrysler of Tri-Cities,BMW,328i,Overhead Camshaft,Manual,Red,21000,99301-3882,Sedan,7659127,Arizona,12,2023
23904,C_CND_023905,2023-12-31,Male,148.5,Chrysler Plymouth,Chevrolet,Metro,DoubleÂ Overhead Camshaft,Auto,Black,31000,53546-9427,Passenger,6030764,Texas,12,2023
